In [1]:
# Etapa 1: Query SQL para histórico ponto

# Caminho físico do lakehouse de destino
path_destino = "abfss://ws_departamento_pessoal@onelake.dfs.fabric.microsoft.com/lk_ponto.Lakehouse/Tables/tab_gold_fato_painel_ponto_historico"

tab_gold_fato_painel_ponto_historico_base = spark.read.format("delta").load(
  "abfss://ws_departamento_pessoal@onelake.dfs.fabric.microsoft.com/lk_ponto.Lakehouse/Tables/tab_gold_fato_painel_ponto_historico_base"
)
tab_gold_fato_painel_ponto_historico_base.createOrReplaceTempView("tab_gold_fato_painel_ponto_historico_base")


df_resultado = spark.sql("""
SELECT
    hb.CODCOLIGADA,
    hb.CHAPA,
    hb.CODPESSOA,
    CASE WHEN df.DATAADMISSAO IS NULL THEN dfu.DATAADMISSAO ELSE df.DATAADMISSAO END AS DATAADMISSAO,
    CASE WHEN df.TIPOADMISSAO IS NULL THEN dfu.TIPOADMISSAO ELSE df.TIPOADMISSAO END AS TIPOADMISSAO,
    CASE WHEN df.DESCRICAO_TIPO_ADMISSAO IS NULL THEN dfu.DESCRICAO_TIPO_ADMISSAO ELSE df.DESCRICAO_TIPO_ADMISSAO END AS DESCRICAO_TIPO_ADMISSAO,
    CASE WHEN df.DATADEMISSAO IS NULL THEN dfu.DATADEMISSAO ELSE df.DATADEMISSAO END AS DATADEMISSAO,
    CASE WHEN df.TIPODEMISSAO IS NULL THEN dfu.TIPODEMISSAO ELSE df.TIPODEMISSAO END AS TIPODEMISSAO,
    CASE WHEN df.DESCRICAO_TIPO_DEMISSAO IS NULL THEN dfu.DESCRICAO_TIPO_DEMISSAO ELSE df.DESCRICAO_TIPO_DEMISSAO END AS DESCRICAO_TIPO_DEMISSAO,
    CASE WHEN df.NOME_FUNCIONARIO IS NULL THEN dfu.NOME_FUNCIONARIO ELSE df.NOME_FUNCIONARIO END AS NOME_FUNCIONARIO,
    CASE WHEN df.CPF IS NULL THEN dfu.CPF ELSE df.CPF END AS CPF,
    CASE WHEN df.DTNASCIMENTO IS NULL THEN dfu.DTNASCIMENTO ELSE df.DTNASCIMENTO END AS DTNASCIMENTO,
    CASE WHEN df.ESTADOCIVIL IS NULL THEN dfu.ESTADOCIVIL ELSE df.ESTADOCIVIL END AS ESTADOCIVIL,
    CASE WHEN df.SEXO IS NULL THEN dfu.SEXO ELSE df.SEXO END AS SEXO,
    CASE WHEN df.PCD IS NULL THEN dfu.PCD ELSE df.PCD END AS PCD,
    CASE WHEN df.CIDADE IS NULL THEN dfu.CIDADE ELSE df.CIDADE END AS CIDADE,
    CASE WHEN df.ESTADO IS NULL THEN dfu.ESTADO ELSE df.ESTADO END AS ESTADO,
    CASE WHEN df.PAIS IS NULL THEN dfu.PAIS ELSE df.PAIS END AS PAIS,
    CASE WHEN df.DESCRICAO_GRAU_INSTRUCAO IS NULL THEN dfu.DESCRICAO_GRAU_INSTRUCAO ELSE df.DESCRICAO_GRAU_INSTRUCAO END AS DESCRICAO_GRAU_INSTRUCAO,
    CASE WHEN df.DESCRICAO_NACIONALIDADE IS NULL THEN dfu.DESCRICAO_NACIONALIDADE ELSE df.DESCRICAO_NACIONALIDADE END AS DESCRICAO_NACIONALIDADE,
    CASE WHEN df.DATAMUDANCA IS NULL THEN dfu.DATAMUDANCA ELSE df.DATAMUDANCA END AS DATAMUDANCA,
    CASE WHEN df.PROXIMA_DATAMUDANCA IS NULL THEN dfu.PROXIMA_DATAMUDANCA ELSE df.PROXIMA_DATAMUDANCA END AS PROXIMA_DATAMUDANCA,
    CASE WHEN df.SITUACAO IS NULL THEN dfu.SITUACAO ELSE df.SITUACAO END AS SITUACAO,
    CASE WHEN df.DESCRICAO_SITUACAO IS NULL THEN dfu.DESCRICAO_SITUACAO ELSE df.DESCRICAO_SITUACAO END AS DESCRICAO_SITUACAO,
    CASE WHEN df.FUNCAO IS NULL THEN dfu.FUNCAO ELSE df.FUNCAO END AS FUNCAO,
    CASE WHEN df.DESCRICAO_FUNCAO IS NULL THEN dfu.DESCRICAO_FUNCAO ELSE df.DESCRICAO_FUNCAO END AS DESCRICAO_FUNCAO,
    CASE WHEN df.SECAO IS NULL THEN dfu.SECAO ELSE df.SECAO END AS SECAO,
    CASE WHEN df.EQUIPE IS NULL THEN dfu.EQUIPE ELSE df.EQUIPE END AS EQUIPE,
    CASE WHEN df.EQUIPE_CC IS NULL THEN dfu.EQUIPE_CC ELSE df.EQUIPE_CC END AS EQUIPE_CC,
    CASE WHEN df.DESCRICAO_EQUIPE IS NULL THEN dfu.DESCRICAO_EQUIPE ELSE df.DESCRICAO_EQUIPE END AS DESCRICAO_EQUIPE,
    CASE WHEN df.UTILIZA_PONTO IS NULL THEN dfu.UTILIZA_PONTO ELSE df.UTILIZA_PONTO END AS UTILIZA_PONTO,
    --mv.CODCOLIGADA,
    hb.CODFILIAL,
    hb.CODCCUSTO                                 AS CCUSTO,
    hb.CODCCUSTO                                 AS CODCCUSTO_GERAL,
    hb.NOME                                      AS NOME_CCUSTO,
    hb.ANOCOMP,
    hb.MESCOMP,
    hb.QF,
    hb.DESCRICAO,
    hb.TIPO,
    hb.HHS_DSR,
    hb.HORAS,
    hb.VLR,
    hb.VLRS_DSR
    
FROM tab_gold_fato_painel_ponto_historico_base AS hb
LEFT JOIN lk_departamento_pessoal.tab_gold_dim_funcionario_ultima_posicao_mes AS df 
                     ON df.CODCOLIGADA = hb.CODCOLIGADA 
                    AND df.CHAPA = hb.CHAPA 
                    AND YEAR(df.DATA) = hb.ANOCOMP AND MONTH(df.DATA) = hb.MESCOMP
LEFT JOIN lk_departamento_pessoal.tab_gold_dim_funcionario_ultima_posicao AS dfu 
                     ON dfu.CODCOLIGADA = hb.CODCOLIGADA 
                    AND dfu.CHAPA = hb.CHAPA 


""")

# Salva no Lakehouse de destino usando caminho físico
df_resultado.write.format("delta").mode("overwrite").save(path_destino)


StatementMeta(, 3930df94-5f16-4e59-932a-cbb8ce5cf04d, 3, Finished, Available, Finished)